In [3]:
import numpy as np 
from classy import Class 
import multiprocessing as mp 
import os
import matplotlib.pyplot as plt 
import scipy.interpolate as sp
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os 
from os import path
import pickle 
import csv
import pandas as pd
from numpy import genfromtxt
import glob 
import shutil
from pathlib import Path
from re import search 

In [4]:
M_sol = 1.989e30 # solar mass in kg
Mpc = 3.086e22 # Mpc in m
c=np.logspace(-11.3,-5, 40)
kvec = np.logspace(-4,np.log10(100),100)
measured_omegacdm=.1188
Sig_8 = 0.811
A_s = 2.1e-9

In [5]:
commonsettings = {
          'N_ncdm':1,
          'use_ncdm_psd_files': 1,
          'm_ncdm': 7100,
          'T_ncdm':0.7,
          'output':'mPk',
          'P_k_max_1/Mpc':100.0,
          # The next line should be uncommented fgor higher precision (but significantly slower running)
          'ncdm_fluid_approximation':3,
          # You may uncomment this line to get more info on the ncdm sector from Class:
          'background_verbose':1,
          'Maximum q':100
         }   

In [6]:

def trap(f,x):
    integral = 0
    for i in range(1,len(f)):
        integral += 0.5 * (f[i] + f[i-1]) * (x[i] - x[i-1])
    return integral


def Bella(k_vec,Pk_vec):
    
    def R(M):
        c = 2.5
        G = 6.67e-11 # m^3 / kg / s^2
        H100 = 100 * (1000/Mpc) # km/s/Mpc -> 1/s
        omegah2 = 0.1188

        rhobar = omegah2 * 3 * H100**2 / (8 * np.pi * G) / M_sol * Mpc**3 # kg / m^3 -> M_sol / Mpc^3
        return (3 * M / (4 * np.pi * rhobar * c**3))**(1/3) # Mpc
    
#    def W(k,R):
#        return 3 / (k * R)**3 * (np.sin(k*R) - 3 * np.cos(k*R))
    def W(k,R):
        if np.isscalar(k):
            if k * R > 1:
                return 0
            else:
                return 1
        else:
            result = np.zeros(len(k))
            for i in range(len(k)):
                if k[i] * R < 1 and k[i+1] * R > 1:
                    result[i] = (1 - k[i] * R)/(k[i+1]*R - k[i]*R)
                elif k[i] * R < 1:
                    result[i] = 1
            return result

    k_vals = np.array(k_vec)
    Pk_vals = np.array(Pk_vec)
    def S(M):
        Rv = R(M)
        
        integrand = k_vals**2 * Pk_vals * W(k_vals, Rv)**2 / (2 * ( np.pi**2))
        return trap(integrand,k_vals)
    
    
    
    M0 = 3.2e12 # M_sol / h
    P_spline  = sp.CubicSpline(k_vals,Pk_vals)
    def dNdlnM(M):
        Rv = R(M)
        return 1 / 44.5 / (6 * np.pi**2) * (M0 / M) / Rv**3 / np.sqrt(2 * np.pi * ( S(M) - S(M0))) * P_spline(1/Rv)
    
    lnM_vals = np.linspace(np.log(1e8),np.log(M0))
#    print("R",R(np.exp(lnM_vals)))
    sv = np.zeros(len(lnM_vals)-1)
    for i in range(len(sv)):
        sv[i] = S(np.exp(lnM_vals[i]))- S(np.exp(lnM_vals[-1]))
    
#    plt.figure()
#    plt.semilogy(lnM_vals[:-1],sv)
#    plt.show()
#    print(sv)
    integrand = np.zeros(len(lnM_vals)-1)
    for i in range(len(integrand)):
        integrand[i] = dNdlnM(np.exp(lnM_vals[i]))
#    print(integrand)
#    plt.figure()
#    plt.semilogy(lnM_vals[:-1],integrand)
#    plt.show()
    return trap(integrand,lnM_vals[:-1])

In [13]:

def ideal_sigma8 (spec_file, omega_h_h):
    
    NH = Class()
        #use method .set() to 
    NH.set(commonsettings)
    othersettings = { 'ncdm_psd_filenames': spec_file,
                      'omega_cdm': measured_omegacdm - omega_h_h, 
                      'omega_ncdm': omega_h_h, 
                      'A_s': 2.1e-9,
                    }

    #use method .compute() to get data for my specific 'Spec-' file
    NH.set(othersettings)
    NH.compute()

    Sigma8_value = NH.sigma8()
    print("",Sigma8_value,"")
    
    ideal_value = ((Sig_8)/(Sigma8_value))**2*(A_s)
    
    NH.struct_cleanup()
    
    return ideal_value

['eps', 'f', 'eps_full', 'f_full', 'omega_s', 'a', 'T', 'Lep']


In [ ]:
def make_Pk(spec_file,omega_h_h):
    
    othersettings = { 'ncdm_psd_filenames': spec_file,
                      'omega_cdm': measured_omegacdm - omega_h_h, 
                      'omega_ncdm': omega_h_h, 
                      'A_s': ideal_sigma8(spec_file, omega_h_h),
                    }
    
    for key, value in othersettings.items():
        print(key, ' : ', value)
    for key, value in commonsettings.items():
        print(key, ' : ', value)
    
    # array of k values in 1/Mpc


    NH = Class()
    NH.set(commonsettings)
    NH.set(othersettings)
    NH.compute()
    
    new_sig8 = NH.sigma8()
    pkNH = [] 
    
        #MPk- Matter Power Spectrum
    for k in kvec:
        pkNH.append(NH.pk(k,0.))
    h = NH.h()
    NH.struct_cleanup()
    

    np.save(spec_file+ '-Pknew',np.array(pkNH))
    np.save(spec_file+'-knew',kvec/h)
    
    return new_sig8

In [ ]:
make_Pk(spec_file, omega_h_h)

In [ ]:
def stack_spec_correctpk(file):
    try:
        top_folder_name = "powerspec" + '/' + "PaperRESULTS copy"  #name of the current directory you want to work in 
        path = file                           #name of the folder that we want to sort, also the input to our function 
        read_me = top_folder_name + "/" + path + "/"

        list_my_path = path.split('x')      #spliting the name of sub-folder from current directory at the letter x 

        epsilon, f = np.loadtxt(read_me + "Eps.csv", delimiter=",", usecols=(0,1), unpack=True) #loading contents of folder 
        a, T, Le, LMu, Ltu = np.loadtxt( read_me + "Lep.csv", delimiter=",", usecols=(0,1,2,3,4), unpack=True)
        omega_h_h = np.load(read_me + "Omegahh.npy")

        if omega_h_h > measured_omegacdm:

            print("Cannot use Omega_s h^2 > {}".format(measured_omegacdm))
            #continue  
            
        np.savetxt(read_me + "Spec", np.column_stack((epsilon,f)))
        spec = np.loadtxt(read_me + 'Spec')

        spec_file = read_me + 'Spec'

        make_Pk(spec_file, omega_h_h) #make corrected k and pk files for all the results in PaperRESULTS copy
        
        os.remove(spec_file)
        
        if os.path.exists(read_me + "Spec-knew.npy"): #next three if statements check if our missing files exist in our path 
            wave_number = np.load(read_me+"Spec-knew.npy")
        else:
            print("Missing_File_k")


        if os.path.exists(read_me + "Spec-Pknew.npy"):
            matter_power_spectrum = np.load(read_me+"Spec-Pknew.npy")
        else:
            print("Missing_File_Pk")

        if os.path.exists(read_me + "N_sh.npy"):
            N_sh =  np.load(read_me +"N_sh.npy")
        else:
            print("Missing_File_sh")

        y = 0 
        for y in range (0,1):  #Run the if else statement once
            if y == 1:
                break 
        if os.path.exists(read_me + "Spec-knew.npy"):
            arraydict1 = {     
                            'DMspectrum': np.array((epsilon,f)),
                            'Lep_num_evo': np.array((a, T, Le, LMu, Ltu)),
                            'matter_pwr_spectrum' : matter_power_spectrum,
                            'Wave_Number':wave_number,
                            'N_s_h' : float(N_sh),
                            'omega_HH' : float(omega_h_h),
                            'r' : float(list_my_path [0]),
                            'Total Inital Lepton Number': float(list_my_path [1]),
                            'Theta^2': float(list_my_path [2]),
                                    }

            np.savez ( file = file + '.npz', arraydict1 = arraydict1)

            file1 = np.load(file + '.npz', allow_pickle= True)
            tmpdict = file1['arraydict1'].item()
            for key in tmpdict: 
                print ((key, tmpdict[key])) 

        else:
            arraydict2 = {
                            'DMspectrum': np.array((epsilon,f)),
                            'Lep_num_evo': np.array((a, T, Le, LMu, Ltu)),
                            'r' : float(list_my_path [0]),
                            'Total Inital Lepton Number': float(list_my_path [1]),
                            'Theta^2': float(list_my_path [2]),
                        }



            np.savez ( file = file + '.npz', arraydict2 = arraydict2)  

            file2 = np.load(file + '.npz', allow_pickle= True)
            tmpdict = file2['arraydict2'].item()
            for key in tmpdict: 
                print ((key, tmpdict[key]))

    except:
        print("Classy stop")
        return

In [ ]:
stack_spec_correctpk('1x0.00095x7.5e-12')

In [ ]:
path = 'powerspec' + '/' + 'PaperRESULTS copy' + '/'
read_me  = path + '1x0.0006x4e-12' + '/'
wave_number = np.load(read_me+"Spec-k.npy")
matter_power_spectrum = np.load(read_me+"Spec-Pk.npy")

new_wave_number = np.load(read_me+"Spec-knew.npy")
new_matter_power_spectrum = np.load(read_me+"Spec-Pknew.npy")

In [ ]:
plt.figure()
plt.loglog(wave_number,matter_power_spectrum)
plt.loglog(new_wave_number,new_matter_power_spectrum, '--')
plt.show()

In [ ]:
def sorting ():
    for x in os.listdir('powerspec' + '/' + 'PaperRESULTS copy'):
        
        if os.path.exists('powerspec' + '/' + 'PaperRESULTS copy' + '/' + ".DS_Store"):
     
            os.remove('powerspec' + '/' + 'PaperRESULTS copy' + '/' + ".DS_Store")
        else:
            pathpath = 'Sorted2' + '/'  
            folder_name = ['R = 1', 'R = 0.1', 'R = 10']
            dest1 = 'Sorted2' + "/" + "/" + "R = 0.1" + '/'
            dest2 = 'Sorted2' + "/" + "/" + "R = 1" + '/'
            dest3 = 'Sorted2' + "/" + "R = 10" + "/"


            for y in range (0,3):
                if not os.path.exists(pathpath + folder_name[y]):
                    os.makedirs(pathpath + folder_name[y])

            stack_spec_correctpk(x)
            
            try:
                
                shutil.move( x + '.npz', pathpath)
            except:
                continue




            for files in pathpath:


                if '0.1' in x and not os.path.exists(dest1 + x + '.npz'): 

                    shutil.move(pathpath + x + '.npz', dest1)
                    break 

                if '10' in x and not os.path.exists(dest3 + x + '.npz'): 

                    shutil.move(pathpath + x + '.npz', dest3)
                    break 


                if '1' in x and not os.path.exists(dest2 + x + '.npz'): 

                    shutil.move(pathpath  + x + '.npz', dest2)
                    break

In [ ]:
sorting()